In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv(r'datasets/dataset_compressible_flow_60M_training_nstep180.csv')

# Extract data from 50 million to 55 million (assuming row indices)
test_df = df.iloc[50_000_000:55_000_000]

# Save as test dataset
test_df.to_csv('datasets/dataset_compressible_flow_5M_test.csv', index=False)

In [1]:
import csv

csv_path = r'datasets\dataset_compressible_flow_60M_training_nstep180.csv'

with open(csv_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    headers = next(reader)
    print("Headings:", headers)

Headings: ['A11', 'A21', 'A31', 'A12', 'A22', 'A32', 'A13', 'A23', 'A33', 'alpha_brf', 'beta_brf', 'gamma_brf', 'ResVort', 'ResStrain', 'Shear', 'Delta', 'lambda_ci', 'Q', 'lambda_2']
